# import

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import accuracy_score

# Открытие данных

In [2]:
df_train = pd.read_csv('/content/drive/MyDrive/всерос_по_ии_24/тренировочный_этап/[G] Определение тональности отзывов на банки/train (1).csv', sep ='\t', index_col=0)#,  on_bad_lines='skip')
df_train

,Score,Text
idx,,
0,Positive,В Альфа-Банке работает замечательная девушка -...
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,Positive,Очень порадовала оперативность работы в банке....
3,Negative,Имела неосторожность оформить потреб. кредит в...
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...


In [3]:
df_test = pd.read_csv('/content/drive/MyDrive/всерос_по_ии_24/тренировочный_этап/[G] Определение тональности отзывов на банки/test (1).csv', sep ='\t', index_col=0)#,  on_bad_lines='skip')
df_test

,Text
idx,
13999,Замечательная история об открытии счета в банк...
14000,"23.07.14г. я досрочно внес сумму, указанную оп..."
14001,Была задолженность по кредитной карте. Банк по...
14002,Срочно понадобилась определенная денежная сумм...
14003,Здравствуйте. В местном отделении Бинбанка г. ...
...,...
19994,Уважаемые представители Русфинанс Банка! Прошу...
19995,04 июня сняли с дебетовой карты банка 10 000 T...
19996,В апреле решил я купить шуруповерт. Посмотрел ...


In [4]:
df_train.isna().sum()

,0
Score,0
Text,0


# Преобразование данных

In [5]:
def encode(x):
    if x == 'Positive': x = 1
    else: x = 0
    return x

for i, x in enumerate(df_train['Score']):
    df_train['Score'][i] = encode(x)

df_train['Score'] = df_train['Score'].astype('int')

df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13999 entries, 0 to 13998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Score   13999 non-null  int64 
 1   Text    13999 non-null  object
dtypes: int64(1), object(1)
memory usage: 844.1+ KB


In [6]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X = vectorizer.fit_transform(df_train['Text'])
X

<13999x106454 sparse matrix of type '<class 'numpy.float64'>'
	with 2171131 stored elements in Compressed Sparse Row format>

In [7]:
y = np.array(df_train['Score'])
y

array([1, 0, 1, ..., 1, 1, 0])

# создание и оценка модели

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

**SVC**

In [ ]:
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
y_pred = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8928571428571429

**gridsearch random forest**

In [9]:
# ps = PredefinedSplit(test_fold=[-1 if i in X_train.index else 0 for i in X.index])
# ps

AttributeError: 'csr_matrix' object has no attribute 'index'

In [10]:
rfc_rs = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid={
        'n_estimators': [20, 50, 100, 200, 250, 300],
        'max_depth': [7, 8, 10, 20, 25, None],
        'max_features': ['sqrt', 'log2', None],
        'criterion': ['gini', 'entropy', 'log_loss'],
        'min_samples_split': [1, 2, 5, 10],
        'n_jobs': [-1]
    },
    scoring='accuracy',
    verbose=52,
    cv=2
)

In [11]:
rfc_rs.fit(X, y)

Fitting 2 folds for each of 1296 candidates, totalling 2592 fits
[CV 1/2; 1/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1
[CV 1/2; 1/1296] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1;, score=nan total time=   0.0s
[CV 2/2; 1/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1
[CV 2/2; 1/1296] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=20, n_jobs=-1;, score=nan total time=   0.0s
[CV 1/2; 2/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=50, n_jobs=-1
[CV 1/2; 2/1296] END criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=50, n_jobs=-1;, score=nan total time=   0.0s
[CV 2/2; 2/1296] START criterion=gini, max_depth=7, max_features=sqrt, min_samples_split=1, n_estimators=50, n_jobs=-1
[CV 2/2; 2/1296

KeyboardInterrupt: 

In [ ]:
y_pred = rfc_rs.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

# создание предикта

In [ ]:
X_pred = vectorizer.transform(df_test['Text'])
pred = clf.predict(X_pred)
pred

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
def re(x):
    if x == 1: x = 'Positive'
    else: x = 'Negative'
    return x

lst = []

for x in pred:
    lst.append(re(x))

In [ ]:
lst.count('Positive')

2911

In [ ]:
lst.count('Negative')

3089

In [ ]:
df_sub = pd.read_csv('/content/drive/MyDrive/всерос_по_ии_24/тренировочный_этап/[G] Определение тональности отзывов на банки/sample_submission.csv', sep ='\t', index_col=0)
df_sub

,Score
idx,
13999,Positive
14000,Positive
14001,Positive
14002,Positive
14003,Positive
...,...
19994,Positive
19995,Positive
19996,Positive


In [ ]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000 entries, 13999 to 19998
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Score   6000 non-null   object
dtypes: object(1)
memory usage: 93.8+ KB


In [ ]:
df_sub['Score'] = lst
df_sub

,Score
idx,
13999,Negative
14000,Negative
14001,Negative
14002,Negative
14003,Negative
...,...
19994,Positive
19995,Positive
19996,Negative


In [ ]:
df_sub.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000 entries, 13999 to 19998
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Score   6000 non-null   object
dtypes: object(1)
memory usage: 93.8+ KB


In [ ]:
df_sub.to_csv('subg_svc.csv', sep ='\t')